In [15]:
import pydgraph
import json

In [8]:
client_stub = pydgraph.DgraphClientStub('localhost:9080')
client = pydgraph.DgraphClient(client_stub)

In [34]:
# Drop All - discard all data and start from a clean slate.
def drop_all(client):
    return client.alter(pydgraph.Operation(drop_all=True))

# Set schema.
def set_schema(client):
    schema = """
    name: string @index(exact) .
    depends: [uid] @reverse .
    version: string .
    type Package {
        name
        depends
        version
    }
    """
    return client.alter(pydgraph.Operation(schema=schema))

drop_all(client)

set_schema(client)



In [36]:
txn = client.txn()

try:
    django = {
        'uid': '_:django',
        'dgraph.type': 'Package',
        'name': 'Django',
        'depends': [
            {
                'uid': '_:pytz',
                'dgraph.type': 'Package',
                'name': 'pytz',
                'version_specifier': '*'
            },
            {
                'uid': '_:sqlparse',
                'dgraph.type': 'Package',
                'name': 'sqlparse',
                'version_specifier': '>=0.2.2'
            },
            {
                'uid': '_:asgiref',
                'dgraph.type': 'Package',
                'name': 'asgiref',
                'version_specifier': '~=3.2'
            },
                        {
                'uid': '_:argon2-cffi',
                'dgraph.type': 'Package',
                'name': 'argon2-cffi',
                'version_specifier': '>=16.1.0'
            },
                        {
                'uid': '_:bcrypt',
                'dgraph.type': 'Package',
                'name': 'bcrypt',
                'version_specifier': '*'
            },
        ],
    }

    asgiref = {
        'uid': '_:asgiref',
        'dgraph.type': 'Package',
        'name': 'asgiref',
        'depends': [
            {
                'uid': '_:pytest',
                'dgraph.type': 'Package',
                'name': 'pytest',
                'version_specifier': '~=4.3.0'
            },
            {
                'uid': '_:pytest-asyncio',
                'dgraph.type': 'Package',
                'name': 'pytest-asyncio',
                'version_specifier': '~=0.10.0'
            },
           
        ],
    }
    
    pytest = {
        'uid': '_:pytest',
        'dgraph.type': 'Package',
        'name': 'pytest',
        'depends': [
            {
                'uid': '_:py',
                'dgraph.type': 'Package',
                'name': 'py',
                'version_specifier': '>=1.5.0'
            },
            {
                'uid': '_:packaging',
                'dgraph.type': 'Package',
                'name': 'packaging',
                'version_specifier': '*'
            },
           
        ],
    }
    
    response = txn.mutate(set_obj=django)
    response = txn.mutate(set_obj=asgiref)
    response = txn.mutate(set_obj=pytest)

    txn.commit()
finally:
  txn.discard()

In [40]:
# Query for data.
def query_package(client, package_name):
    # Run query.
    query = """query all($a: string) {
        all(func: eq(name, $a)) {
            uid
            name
            version
            depends {
                name
                version
                version_specifier
            }
        }
    }"""

    variables = {'$a': package_name}
    res = client.txn(read_only=True).query(query, variables=variables)
    packages = json.loads(res.json)

    # Print results.
    print(packages)


In [39]:

query_package(client, 'pytest')

{'all': [{'uid': '0x2c', 'name': 'pytest'}, {'uid': '0x2e', 'name': 'pytest', 'depends': [{'name': 'py', 'version_specifier': '>=1.5.0'}, {'name': 'packaging', 'version_specifier': '*'}]}]}
